<a href="https://colab.research.google.com/github/TheRadDani/graph-link-prediction-with-FiLM/blob/main/FiLM_for_link_prediction_on_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install torch_geometric

In [11]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.nn import FiLMConv

In [12]:
path = os.path.join(os.getcwd(), 'data', 'PPI')
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [13]:
class Network(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
               dropout=0.0):
    super(Network, self).__init__()

    self.dropout = dropout
    self.convs = nn.ModuleList()
    self.convs.append(FiLMConv(in_channels, hidden_channels))
    for _ in range(num_layers - 2):
      self.convs.append(FiLMConv(hidden_channels, hidden_channels))
    self.convs.append(FiLMConv(hidden_channels, out_channels, act=None))

    self.norms = nn.ModuleList()
    for _ in range(num_layers - 1):
      self.norms.append(nn.LayerNorm(hidden_channels))

  def forward(self, x, edge_index):
    for conv, norm in zip(self.convs[:-1], self.norms):
      x = norm(conv(x, edge_index))
      x = F.dropout(x, p=self.dropout, training=self.training)
    x = self.convs[-1](x, edge_index)
    return x

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Network(in_channels=train_dataset.num_features,hidden_channels=320,
                out_channels=train_dataset.num_classes, num_layers = 4,
                dropout=0.1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
next(iter(train_dataloader))

DataBatch(x=[4561, 50], edge_index=[2, 117636], y=[4561, 121], batch=[4561], ptr=[3])

In [30]:
def train():
  model.train()
  total_loss = 0

  for data in train_dataloader:
    data = data.to(device)
    optimizer.zero_grad()
    loss= criterion(model(data.x, data.edge_index), data.y)
    total_loss += loss.item() * data.num_graphs
    loss.backward()
    optimizer.step()
  return total_loss / len(train_dataloader.dataset)

@torch.no_grad()
def test(loader):
  model.eval()
  ys, preds = list(), list()
  for data in loader:
    data = data.to(device)
    ys.append(data.y)
    out = model(data.x, data.edge_index)
    preds.append((out > 0).float())
  y , pred = torch.cat(ys, dim=0).cpu().numpy(), torch.cat(preds, dim=0).cpu().numpy()
  return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0

In [35]:
for epoch in range(500):
  loss = train()
  val_f1 = test(val_dataloader)
  test_f1 = test(test_dataloader)
  print(f'Epoch: {epoch+1}, Loss: {loss:.4f}, \
   Val F1: {val_f1:.4f}, Test F1: {test_f1:.4f}')

Epoch: 1, Loss: 0.0077,    Val F1: 0.9870, Test F1: 0.9926
Epoch: 2, Loss: 0.0079,    Val F1: 0.9870, Test F1: 0.9925
Epoch: 3, Loss: 0.0082,    Val F1: 0.9869, Test F1: 0.9926
Epoch: 4, Loss: 0.0085,    Val F1: 0.9870, Test F1: 0.9925
Epoch: 5, Loss: 0.0081,    Val F1: 0.9869, Test F1: 0.9925
Epoch: 6, Loss: 0.0081,    Val F1: 0.9869, Test F1: 0.9925
Epoch: 7, Loss: 0.0087,    Val F1: 0.9869, Test F1: 0.9923
Epoch: 8, Loss: 0.0085,    Val F1: 0.9867, Test F1: 0.9922
Epoch: 9, Loss: 0.0084,    Val F1: 0.9871, Test F1: 0.9925
Epoch: 10, Loss: 0.0087,    Val F1: 0.9870, Test F1: 0.9925
Epoch: 11, Loss: 0.0085,    Val F1: 0.9871, Test F1: 0.9925
Epoch: 12, Loss: 0.0090,    Val F1: 0.9868, Test F1: 0.9925
Epoch: 13, Loss: 0.0089,    Val F1: 0.9870, Test F1: 0.9923
Epoch: 14, Loss: 0.0086,    Val F1: 0.9869, Test F1: 0.9923
Epoch: 15, Loss: 0.0081,    Val F1: 0.9871, Test F1: 0.9925
Epoch: 16, Loss: 0.0080,    Val F1: 0.9872, Test F1: 0.9924
Epoch: 17, Loss: 0.0081,    Val F1: 0.9870, Test 